In [252]:
# imports
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from xgboost import XGBRegressor
from sklearn.inspection import permutation_importance

In [253]:
# options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [254]:
# let's read in the data and take a look
X_full = pd.read_csv('train.csv', index_col='Id')
X_test_full = pd.read_csv('test.csv', index_col='Id')
X_full.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,NaN,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,NaN,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [255]:
X_test_full.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,NaN,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [256]:
# check shapes for reference
print(X_full.shape)
print(X_test_full.shape)

(1460, 80)
(1459, 79)


In [257]:
# let's split the training data into our data and target
X_full.dropna(subset='SalePrice', axis=0, inplace=True)
y = np.log(X_full['SalePrice']) # log for evaluation
X_full.drop('SalePrice', axis=1, inplace=True)

In [258]:
# we could use train_test_split, but i am going to use CV

In [259]:
# let's split up our categorical and numerical columns

ordinal_qual_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']

ordinal_cols = ordinal_qual_cols

onehot_cols = [col for col in X_full.columns if 
                   (X_full[col].dtype == 'object') and 
                   (col not in ordinal_cols)]

categorical_cols = ordinal_cols + onehot_cols

numerical_cols = [col for col in X_full.columns if 
                  (X_full[col].dtype in ['int64','float64'])]

print(categorical_cols)
print(numerical_cols)

cols = categorical_cols + numerical_cols

X_train = X_full[cols].copy()
X_test = X_test_full[cols].copy()

['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'GarageType', 'GarageFinish', 'PavedDrive', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'Enclos

In [260]:
# let's figure out how we want to process our data
X_train.head()

,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond,PoolQC,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,CentralAir,Electrical,Functional,GarageType,GarageFinish,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Gd,TA,Gd,TA,Ex,Gd,NaN,TA,TA,NaN,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,No,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,NaN,NaN,WD,Normal,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008
2,TA,TA,Gd,TA,Ex,TA,TA,TA,TA,NaN,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,NaN,CBlock,Gd,ALQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,NaN,NaN,WD,Normal,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007
3,Gd,TA,Gd,TA,Ex,Gd,TA,TA,TA,NaN,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,Mn,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,NaN,NaN,WD,Normal,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008
4,TA,TA,TA,Gd,Gd,Gd,Gd,TA,TA,NaN,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,NaN,BrkTil,No,ALQ,Unf,GasA,Y,SBrkr,Typ,Detchd,Unf,Y,NaN,NaN,WD,Abnorml,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006
5,Gd,TA,Gd,TA,Ex,Gd,TA,TA,TA,NaN,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,Av,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,NaN,NaN,WD,Normal,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008


In [261]:
# we can likely let one hot encoding handle all of these nulls
# X_train[categorical_cols].isna().sum()

In [262]:
# LotFrontage, GarageYrBlt, and MasVnrArea only ones with na. lets look further in
# X_train[numerical_cols].isna().sum()

In [263]:
# X_train[X_train['LotFrontage'].isna()].head()
# X_train[X_train.LotFrontage == 0]
# it seems like any NA values in lotfrontage is if the house doesn't have any
# we can set this column's na values to 0 most likely

In [264]:
# X_train[X_train['GarageYrBlt'].isna()].head()
# it seems like any na values in this column are when there is no garage
# X_train.head()
# the garage dates correlate pretty well with when the house was built
# i think dropping the garage year built and instead adding a hasGarage will be best

In [265]:
# X_train[X_train['MasVnrArea'].isna()].head()
# it seems like an NaN value means there is not one, so we can input 0 for the missing values here

In [266]:
# no cardinality is too high, so no need to drop any categorical
# [X_train[col].nunique() for col in X_train[categorical_cols].columns]

In [267]:
# let's look into the testing data too to make sure these transformations are ok
# the one hot seems like it will work here, but a has pool feature should be added
# X_test[categorical_cols].isna().sum()

In [268]:
# it appears that all of these NA values would also likely be best replaced with a 0
# X_test[numerical_cols].isna().sum()

In [269]:
# the cardinalities are roughly the same, so one hot will still be the move
# [X_test[col].nunique() for col in X_test[categorical_cols].columns]

In [270]:
# now that i know what the data is like, i will start feature engineering

for df in [X_train, X_test]:
    df['TotalSF'] = (df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF'])
    df['TotalBath'] = (df['BsmtHalfBath'] * .5 + df['BsmtFullBath'] + df['FullBath'] + df['HalfBath'] * .5)
    df['SellAge'] = (df['YrSold'] - df['YearBuilt'])
    df['OverallRating'] = (df['OverallQual'] + df['OverallCond'])

if 'TotalSF' not in numerical_cols:
    numerical_cols.extend(['TotalSF',
                           'TotalBath',
                           'SellAge',
                           'OverallRating',
                          ])

if 'GarageYrBlt' in numerical_cols:
    numerical_cols.remove('GarageYrBlt')
    numerical_cols.remove('BsmtFullBath')
    numerical_cols.remove('BsmtHalfBath')
    numerical_cols.remove('FullBath')
    numerical_cols.remove('HalfBath')

categorical_cols = ordinal_cols + onehot_cols
cols = categorical_cols + numerical_cols
X_train = X_train[cols].copy()
X_test = X_test[cols].copy()
X_train.head()

,ExterQual,ExterCond,BsmtQual,BsmtCond,HeatingQC,KitchenQual,FireplaceQu,GarageQual,GarageCond,PoolQC,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,Foundation,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,CentralAir,Electrical,Functional,GarageType,GarageFinish,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,TotalSF,TotalBath,SellAge,OverallRating
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Gd,TA,Gd,TA,Ex,Gd,NaN,TA,TA,NaN,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,No,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,NaN,NaN,WD,Normal,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,3,1,8,0,2,548,0,61,0,0,0,0,0,2,2008,2566,3.5,5,12
2,TA,TA,Gd,TA,Ex,TA,TA,TA,TA,NaN,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,NaN,CBlock,Gd,ALQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,NaN,NaN,WD,Normal,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,3,1,6,1,2,460,298,0,0,0,0,0,0,5,2007,2524,2.5,31,14
3,Gd,TA,Gd,TA,Ex,Gd,TA,TA,TA,NaN,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,Mn,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,NaN,NaN,WD,Normal,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,3,1,6,1,2,608,0,42,0,0,0,0,0,9,2008,2706,3.5,7,12
4,TA,TA,TA,Gd,Gd,Gd,Gd,TA,TA,NaN,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,NaN,BrkTil,No,ALQ,Unf,GasA,Y,SBrkr,Typ,Detchd,Unf,Y,NaN,NaN,WD,Abnorml,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,3,1,7,1,3,642,0,35,272,0,0,0,0,2,2006,2473,2.0,91,12
5,Gd,TA,Gd,TA,Ex,Gd,TA,TA,TA,NaN,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,PConc,Av,GLQ,Unf,GasA,Y,SBrkr,Typ,Attchd,RFn,Y,NaN,NaN,WD,Normal,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,4,1,9,1,3,836,192,84,0,0,0,0,0,12,2008,3343,3.5,8,13


In [271]:
# i can begin creating the pipeline now

quality_order = ['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']

ordinal_qual_transformer = Pipeline(steps=[
    ('ordqual', OrdinalEncoder(categories = [quality_order]*len(ordinal_qual_cols), handle_unknown='use_encoded_value', unknown_value=-1))
])

onehot_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='constant', fill_value=0))
])

preprocessor = ColumnTransformer(transformers=[
    ('ordq', ordinal_qual_transformer, ordinal_qual_cols),
    ('oh', onehot_transformer, onehot_cols),
    ('num', numerical_transformer, numerical_cols)
])

In [272]:
# it is time to create the model in the pipeline

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(#n_estimators=450, 
                           #max_depth=2, 
                           #learning_rate=.2, 
                           #colsample_bytree=.6,
                           #alpha=.1,
                           random_state=1))
])

In [273]:
# now i will use grid search to find the best parameters for the model

param_grid={
    'model__n_estimators':[621],
    'model__max_depth':[2],
    'model__learning_rate':[.125],
    'model__colsample_bytree':[.525],
    'model__alpha':[.125],
    'model__lambda':[1.1],
}

#n_estimators=621, max_depth=2, learning_rate=.125, colsample_bytree=.525, alpha=.125, lambda=1.1

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y)
best_pipeline = grid_search.best_estimator_
grid_search.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END model__alpha=0.125, model__colsample_bytree=0.525, model__lambda=1.1, model__learning_rate=0.125, model__max_depth=2, model__n_estimators=621; total time=   0.2s
[CV] END model__alpha=0.125, model__colsample_bytree=0.525, model__lambda=1.1, model__learning_rate=0.125, model__max_depth=2, model__n_estimators=621; total time=   0.2s
[CV] END model__alpha=0.125, model__colsample_bytree=0.525, model__lambda=1.1, model__learning_rate=0.125, model__max_depth=2, model__n_estimators=621; total time=   0.2s
[CV] END model__alpha=0.125, model__colsample_bytree=0.525, model__lambda=1.1, model__learning_rate=0.125, model__max_depth=2, model__n_estimators=621; total time=   0.1s
[CV] END model__alpha=0.125, model__colsample_bytree=0.525, model__lambda=1.1, model__learning_rate=0.125, model__max_depth=2, model__n_estimators=621; total time=   0.2s
[CV] END model__alpha=0.125, model__colsample_bytree=0.525, model__lambda=1.1, model__

{'model__alpha': 0.125,
 'model__colsample_bytree': 0.525,
 'model__lambda': 1.1,
 'model__learning_rate': 0.125,
 'model__max_depth': 2,
 'model__n_estimators': 621}

In [274]:
# use cv to see how strong pipeline is

cv_scores = -1 * cross_val_score(
    best_pipeline,
    X_train,
    y,
    cv=5,
    scoring='neg_root_mean_squared_error'
)

cv_scoresr2 = cross_val_score(
    best_pipeline,
    X_train,
    y,
    cv=5,
    scoring='r2'
)

print("RMSE per fold:", cv_scores)
print("Average RMSE:", cv_scores.mean())
print("R^2 per fold:", cv_scoresr2)
print("Average R^2:", cv_scoresr2.mean())

RMSE per fold: [0.10849764 0.12933197 0.12463484 0.1029219  0.12460795]
Average RMSE: 0.11799886187398742
R^2 per fold: [0.92121831 0.90744859 0.90742677 0.92679837 0.89849001]
Average R^2: 0.9122764096200744


Best submitted (on Kaggle, slight differences when run here) - RMSE: 0.11641603498024163, R^2: 0.9146652682262104 - bathrooms out of df, params n_estimators=621, max_depth=2, learning_rate=.125, colsample_bytree=.525, alpha=.125, lambda=1.1

In [275]:
# using permutation importance to see what the most/least important features are

result = permutation_importance(best_pipeline,
                                X_train,
                                y,
                                n_repeats=5,
                                random_state=1,
                                scoring='neg_root_mean_squared_error'
                               )

importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance_mean': result.importances_mean,
    'importance_std': result.importances_std
}).sort_values(by='importance_mean', ascending=False)

print(importance_df.head(10))
print(importance_df.tail(10))

          feature  importance_mean  importance_std
74        TotalSF         0.113024        0.001896
46    OverallQual         0.028741        0.001079
77  OverallRating         0.026921        0.001240
75      TotalBath         0.019933        0.000725
76        SellAge         0.016078        0.000782
58      GrLivArea         0.015206        0.000655
45        LotArea         0.014518        0.000720
18   Neighborhood         0.014200        0.000469
10       MSZoning         0.011578        0.000719
63     GarageCars         0.010795        0.000636
        feature  importance_mean  importance_std
68    3SsnPorch         0.000136        0.000028
43   MSSubClass         0.000128        0.000041
23    RoofStyle         0.000101        0.000034
17    LandSlope         0.000068        0.000027
32      Heating         0.000033        0.000010
71      MiscVal         0.000000        0.000000
40  MiscFeature         0.000000        0.000000
20   Condition2         0.000000        0.00000

In [276]:
# once satisfied with the MAE, we can predict the testing data and submit
best_pipeline.fit(X_train, y)

predictions = np.exp(best_pipeline.predict(X_test)) # re-exponent from beginning log

submission = pd.DataFrame({
    'Id':X_test.index,
    'SalePrice':predictions
})
submission.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()

,Id,SalePrice
0,1461,116915.484
1,1462,160574.330
2,1463,184744.480
3,1464,191240.000
4,1465,187340.340
